# Control Component: PID Controller

The `PIDController` is a core component for implementing feedback control loops. It calculates a control action based on the error between a desired `setpoint` and a measured `process_variable`. It is a versatile controller used in a vast majority of industrial control systems.

This implementation includes:
- **Proportional (P)**: Reacts to the current error.
- **Integral (I)**: Accumulates past errors to eliminate steady-state error.
- **Derivative (D)**: Responds to the rate of change of the error to dampen oscillations.
- **Anti-Windup**: Prevents the integral term from growing uncontrollably when the controller output is saturated at its maximum or minimum limit.

## Simulation Example

To demonstrate the PID controller, we will simulate a simple first-order process. Imagine this is a tank of water where the `process_variable` is the temperature, and the `control_action` is the power applied to a heater. The tank also loses heat to the environment.

The goal is to use the PID controller to bring the temperature to a setpoint and maintain it there. We will also change the setpoint midway through the simulation to observe how the controller adapts.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from swp.local_agents.control.pid_controller import PIDController

# 1. Create a PIDController instance
pid = PIDController(
    Kp=5.0, 
    Ki=0.5, 
    Kd=2.0, 
    setpoint=50.0, # Initial target temperature
    min_output=0.0,  # Heater power cannot be negative
    max_output=100.0 # Max heater power
)

# 2. Simulation settings
dt = 1.0
simulation_duration = 200
num_steps = int(simulation_duration / dt)
history = []

# 3. Simple process simulation
process_variable = 20.0 # Initial temperature
ambient_temp = 20.0
heat_loss_coeff = 0.1

for t in range(num_steps):
    # Change the setpoint at t=100
    if t == 100:
        pid.set_setpoint(75.0)
        
    # Controller computes the action
    observation = {'process_variable': process_variable}
    control_action = pid.compute_control_action(observation, dt)
    
    # Simulate the process: temperature changes based on heater input and heat loss
    heat_loss = (process_variable - ambient_temp) * heat_loss_coeff
    process_variable += (control_action - heat_loss) * dt * 0.1 # 0.1 is a process gain
    
    # Store results
    history.append({
        'time': t * dt,
        'setpoint': pid.setpoint,
        'process_variable': process_variable,
        'control_action': control_action
    })

print("PID controller simulation complete.")

## Results and Visualization

The plots show the `process_variable` (temperature) converging on the `setpoint`. Note the initial overshoot, which is characteristic of PID tuning. When the setpoint is changed at t=100, the controller again drives the process variable to the new target. The `control_action` plot shows the heater power being adjusted by the controller.

In [ ]:
# Create a DataFrame from history
df = pd.DataFrame(history)

print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Process Variable and Setpoint
ax1.plot(df['time'], df['process_variable'], label='Process Variable (Temperature)')
ax1.plot(df['time'], df['setpoint'], label='Setpoint', color='red', linestyle='--')
ax1.set_ylabel('Temperature (°C)')
ax1.set_title('PID Controller Simulation')
ax1.grid(True)
ax1.legend()

# Control Action
ax2.plot(df['time'], df['control_action'], label='Control Action (Heater Power)', color='green')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Power (%)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()